<a href="https://colab.research.google.com/github/poandpo/week3/blob/main/%5B7%ED%8C%80_%EC%9E%84%EC%B1%84%EC%9D%80%5D_Week3_1_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week3_1 Assignment

## [BASIC](#Basic) 
- 토크나이징이 완료된 위키 백과 코퍼스를 다운받고 **단어 사전을 구축하는 함수를 구현**할 수 있다.
- `Skip-Gram` 방식의 학습 데이터 셋을 생성하는 **Dataset과 Dataloader 클래스를 구현**할 수 있다.
- **Negative Sampling** 함수를 구현할 수 있다. 


## [CHALLENGE](#Challenge)
- Skip-Gram을 학습 과정 튜토리얼을 따라하며, **Skip-Gram을 학습하는 클래스를 구현**할 수 있다. 


## [ADVANCED](#Advanced)
- Skip-Gram 방식으로 word embedding을 학습하는 **Word2Vec 클래스를 구현**하고 실제로 학습할 수 있다.
- 학습이 완료된 word embedding을 불러와 **Gensim 패키지를 사용해 유사한 단어**를 뽑을 수 있다. 

### Reference
- [Skip-Gram negative sampling 한국어 튜토리얼](https://wikidocs.net/69141)
    - (참고) 위 튜토리얼에서는 target word와 context word 페어의 레이블은 1로, target word와 negative sample word 페어의 레이블은 0이 되도록 학습 데이터를 구현해 binary classification을 구현한다. 하지만 우리는 word2vec 논문 방식을 그대로 따르기 위해 label을 생성하지 않고 대신 loss 함수를 변행해서 binary classification을 학습할 것이다. 

In [ ]:
import os 
import sys
import pandas as pd
import numpy as np
import re
from typing import List, Dict
import random

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 12.3 MB/s 
     |████████████████████████████████| 596 kB 50.4 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 6.5 MB 58.0 MB/s 
     |████████████████████████████████| 895 kB 45.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
# seed
seed = 7777
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [ ]:
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


## Basic

### 토크나이징이 완료된 위키 백과 코퍼스 다운로드 및 불용어 사전 크롤링
- 나의 구글 드라이브에 데이터를 다운받아 영구적으로 사용할 수 있도록 하자. 
    - [데이터 다운로드 출처](https://ratsgo.github.io/embedding/downloaddata.html)
- 다운받은 데이터는 토크나이징이 완료된 상태이지만 불용어를 포함하고 있다. 따라서 향후 불용어를 제거하기 위해 불용어 사전을 크롤링하자. 
    - [불용어 사전 출처](https://www.ranks.nl/stopwords/korean)

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/MoneyManager/report/tokenized/

/content/drive/MyDrive/MoneyManager/report/tokenized


In [ ]:
# 데이터 다운로드
!pip install gdown
!gdown https://drive.google.com/u/0/uc?id=1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx
!unzip tokenized.zip

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/u/0/uc?id=1Ybp_DmzNEpsBrUKZ1-NoPDzCMO39f-fx 

unzip:  cannot find or open tokenized.zip, tokenized.zip.zip or tokenized.zip.ZIP.


In [ ]:
# 한국어 불용어 리스트 크롤링
import requests
from bs4 import BeautifulSoup

url = "https://www.ranks.nl/stopwords/korean"
response = requests.get(url, verify = False)

if response.status_code == 200:
    soup = BeautifulSoup(response.text,'html.parser')
    content = soup.select_one('#article178ebefbfb1b165454ec9f168f545239 > div.panel-body > table > tbody > tr')
    stop_words=[]
    for x in content.strings:
        x=x.strip()
        if x:
            stop_words.append(x)
    print(f"# Korean stop words: {len(stop_words)}")
else:
    print(response.status_code)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


# Korean stop words: 677


In [ ]:
stop_words[0]

'아'

In [ ]:
# 코퍼스 로드
docs = []
doc = pd.read_table('wiki_ko_mecab.txt').values.tolist()
for doc in doc:
    docs.extend(doc)

In [ ]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 311,236


In [ ]:
# 문서 개수를 500개로 줄임
docs= random.sample(docs,500)

In [ ]:
print(f"# wiki documents: {len(docs):,}")

# wiki documents: 500


### 단어 사전 구축 함수 구현 
- 문서 리스트를 입력 받아 사전을 생성하는 `make_vocab()` 함수를 구현하라.
- 함수 정의
    - 입력 매개변수
        - docs : 문서 리스트
        - min_count : 최소 단어 등장 빈도수 (단어 빈도가 `min_count` 미만인 단어는 사전에 포함하지 않음)
    - 조건
        - 문서 길이 제한
            - 단어 개수가 3개 이하인 문서는 처리하지 않음. (skip)
        - 사전에 포함되는 단어 빈도수 제한
            - 단어가 빈도가 `min_count` 미만은 단어는 사전에 포함하지 않음.
        - 불용어 제거 
            - 불용어 리스트에 포함된 단어는 제거 
    - 반환값 
        - word2count : 단어별 빈도 사전 (key: 단어, value: 등장 횟수)
        - wid2word : 단어별 인덱스(wid) 사전 (key: 단어 인덱스(int), value: 단어)
        - word2wid : 인덱스(wid)별 단어 사전 (key: 단어, value: 단어 인덱스(int))

In [ ]:
# 문서 내 숫자, 영어 대소문자, 특수문자를 제거 (re package 사용)

docs = [re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', doc) for doc in docs]

In [ ]:
print(f"Check : {docs[0][:1000]}")

Check : 아효 부인 박 씨            는 신라 의 공주 이 자 황후 로  신라 의  대 왕 인 남 해 차차웅      의 딸 이 며 유리 이사금  박환      내 로 갈문왕     의 누이 였 다   대 탈해 이사금 의 왕비 이 기 도 하 다  성 은 박    씨 이 다  아니 부인       아로 부인      이 라고 도 한다  신라 제  대 왕 남해왕      과 그 비인 운제 부인      사이 에 맏 딸 로 태어났 다  남 해 차차웅 은 석탈해 의 재능 을 알아보 고 그 에게 맏 딸 인 아효 부인 을 시집보냈 으며 왕위 를 계승 하 게 하 였 다  그러나 석탈해 는 유리 이사금 에게 지위 를 양보 하 였 고 유리 이사금 의 천거 로 다음 왕위 를 계승 하 게 되 었 다    년 남편 탈해 이사금 이 왕 으로 즉위 하 면서 왕비 에 책봉 되 었 다  오랫동안 아들 이 없 자 탈해 이사금 이 시림 에서 김알지 를 발견 하 여 양자 로 정하 면서 김알지 의 양어머니 가 되 었 다  그러나 뒤 에 아들 석구 추가 태어났 다  증조모 사소부인      선도 산 성모  조부 혁거세 거서간     조모 알영부인     부왕 남 해 차차웅      모후 운제 부인       오빠 유리 이사금         남동생 박환      남동생 내 로 갈문왕        남편 탈해이사금         아들 강조      갈문왕 에 추봉  아들 석 구추       아들 석구 광  양아들 김알지      또는 성한 김수로 문화방송 주말 특별 기획 드라마        년      년  배우  강별  남 해 차차웅 탈해 이사금 박특 알영부인 유리 이사금 분류  신라 의 왕족 분류  신라 의 왕비 분류  탈해 이사금


In [ ]:
def make_vocab(docs:List[str], min_count:int):
    """
    'docs'문서 리스트를 입력 받아 단어 사전을 생성.
    
    return 
        - word2count : 단어별 빈도 사전
        - wid2word : 단어별 인덱스(wid) 사전 
        - word2wid : 인덱스(wid)별 단어 사전
    """

    word2count = dict()
    word2id = dict()
    id2word = dict()
    _word2count = dict()
    
    for doc in tqdm(docs):
        word_list = doc.split()

         # 1. 문서 길이 제한 
        if len(word_list) >= 3 :
         # 2. 임시 딕셔너리(_word2count)에 단어별 등장 빈도 기록
          
          for word in word_list:
            count= _word2count.get(word, 0)
            _word2count[word] = count + 1

        # 3. 불용어 제거
          for word in _word2count.copy().keys():
            if word in stop_words:
              del _word2count[word]
        else:
          pass    

    # 4. 토큰 최소 빈도를 만족하는 토큰만 사전에 추가
    for word,va in _word2count.items():
      if va >= min_count:
       word2count[word]= va
       word2id[word] = len(word2id)
       id2word[len(id2word)] = word

    
    return word2count, word2id, id2word

In [ ]:
word2count, word2id, id2word = make_vocab(docs, min_count=5)

100%|██████████| 500/500 [01:19<00:00,  6.29it/s]


In [ ]:
doc_len = sum(word2count.values()) # 문서 내 모든 단어의 개수 (단어별 등장 빈도의 총 합)
print(f"{doc_len:,}")

143,393


In [ ]:
print(f"# unique word : {len(word2id):,}")

# unique word : 5,055


### Dataset 클래스 구현
- Skip-Gram 방식의 학습 데이터 셋(`Tuple(target_word, context_word)`)을 생성하는 `CustomDataset` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - docs: 문서 리스트
        - word2id: 단어별 인덱스(wid) 사전 (key: 단어 인덱스(int), value: 단어)
        - window_size: Skip-Gram의 윈도우 사이즈
    - 메소드
        - `make_pair()`
            - 문서를 단어로 쪼개고, 사전에 존재하는 단어들만 단어 인덱스로 변경
            - Skip-gram 방식의 `(target_word, context_word)` 페어(tuple)들을 `pairs` 리스트에 담아 반환
        - `__len__()`
            - `pairs` 리스트의 개수 반환
        - `__getitem__(index)`
            - `pairs` 리스트를 인덱싱
    - 주의 사항
        - `nn.Module`를 부모 클래스로 상속 받음 


In [ ]:
class CustomDataset(Dataset):
    """
    문서 리스트를 받아 skip-gram 방식의 (target_word, context_word) 데이터 셋을 생성
    """
    # word2id key: 단어 인덱스(int), value: 단어)
    def __init__(self, docs:List[str], word2id:Dict[str,int], window_size:int=5):
        self.docs = docs
        self.word2id = word2id
        self.window_size = window_size
        self.pairs = self.make_pair()
    
    def make_pair(self):
        """
        (target, context) 형식의 Skip-gram pair 데이터 셋 생성 
        """
        pairs = []

        for doc in self.docs: 
          word_list = doc.split()
          indexs = [self.word2id[word] for word in word_list if word in self.word2id.keys()]
          max= len(indexs) -1

          for idx, index in enumerate(indexs):

            for  i in range(1, self.window_size+1):              
               be_context, af_context = idx-i, idx+i 

               if  be_context >= 0 and af_context <= max:
                  pairs.append((index, indexs[af_context]))
                  pairs.append((index, indexs[be_context]))

               elif be_context >= 0 and af_context > max  :
                  pairs.append((index, indexs[be_context]))

               elif be_context < 0 and af_context <= max:
                  pairs.append((index, indexs[af_context]))

               elif be_context < 0 and af_context > max:
                  pass

        return pairs
        
    def __len__(self):
        return len(self.pairs)
    
    def __getitem__(self, idx):
        return self.pairs[idx]

In [ ]:
dataset = CustomDataset(docs, word2id, window_size=5)

In [ ]:
len(dataset)

1418930

In [ ]:
dataset[0]

(0, 1)

In [ ]:
# verify (target word, context word)
for i, pair in enumerate(dataset):
    if i==100:
        break
    print(f"({id2word[pair[0]]}, {id2word[pair[1]]})")
    

(부인, 박)
(부인, 씨)
(부인, 는)
(부인, 신라)
(부인, 공주)
(박, 씨)
(박, 부인)
(박, 는)
(박, 신라)
(박, 공주)
(박, 신라)
(씨, 는)
(씨, 박)
(씨, 신라)
(씨, 부인)
(씨, 공주)
(씨, 신라)
(씨, 대)
(는, 신라)
(는, 씨)
(는, 공주)
(는, 박)
(는, 신라)
(는, 부인)
(는, 대)
(는, 왕)
(신라, 공주)
(신라, 는)
(신라, 신라)
(신라, 씨)
(신라, 대)
(신라, 박)
(신라, 왕)
(신라, 부인)
(신라, 인)
(공주, 신라)
(공주, 신라)
(공주, 대)
(공주, 는)
(공주, 왕)
(공주, 씨)
(공주, 인)
(공주, 박)
(공주, 남)
(공주, 부인)
(신라, 대)
(신라, 공주)
(신라, 왕)
(신라, 신라)
(신라, 인)
(신라, 는)
(신라, 남)
(신라, 씨)
(신라, 해)
(신라, 박)
(대, 왕)
(대, 신라)
(대, 인)
(대, 공주)
(대, 남)
(대, 신라)
(대, 해)
(대, 는)
(대, 딸)
(대, 씨)
(왕, 인)
(왕, 대)
(왕, 남)
(왕, 신라)
(왕, 해)
(왕, 공주)
(왕, 딸)
(왕, 신라)
(왕, 며)
(왕, 는)
(인, 남)
(인, 왕)
(인, 해)
(인, 대)
(인, 딸)
(인, 신라)
(인, 며)
(인, 공주)
(인, 유리)
(인, 신라)
(남, 해)
(남, 인)
(남, 딸)
(남, 왕)
(남, 며)
(남, 대)
(남, 유리)
(남, 신라)
(남, 이사금)
(남, 공주)
(해, 딸)
(해, 남)
(해, 며)
(해, 인)
(해, 유리)


### 위에서 생성한 `dataset`으로 DataLoader  객체 생성
- `DataLoader` 클래스로 `train_dataloader`객체를 생성하라. 
    - 생성자 매개변수와 값
        - dataset = 위에서 생성한 dataset
        - batch_size = 64
        - shuffle = True

In [ ]:
train_dataloader = DataLoader(dataset, batch_size = 64, shuffle = True)

In [ ]:
len(train_dataloader)

22171

### Negative Sampling 함수 구현
- Skip-Gram은 복잡도를 줄이기 위한 방법으로 negative sampling을 사용한다. 
- `sample_table`이 다음과 같이 주어졌을 때, sample_table에서 랜덤으로 값을 뽑아 (batch_size, n_neg_sample) shape의 matrix를 반환하는 `get_neg_v_negative_sampling()`함수를 구현하라. 
- Sample Table은 negative distribution을 따른다. 
    - [negative distribution 설명](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#How-are-negative-samples-drawn?)
- 함수 정의
    - 입력 매개변수
        - batch_size : 배치 사이즈, matrix의 row 개수 
        - n_neg_sample : negative sample의 개수, matrix의 column 개수
    - 반환값 
        - neg_v : 추출된 negative sample (2차원의 리스트)


In [ ]:
# negative sample을 추출할 sample table 생성 (해당 코드를 참고)
sample_table = []
sample_table_size = doc_len

# noise distribution 생성
alpha = 3/4
frequency_list = np.array(list(word2count.values())) ** alpha

Z = sum(frequency_list)

ratio = frequency_list/Z

negative_sample_dist = np.round(ratio*sample_table_size)

for wid, c in enumerate(negative_sample_dist):
    sample_table.extend([wid]*int(c))

In [ ]:
len(sample_table)

143204

In [ ]:
def get_neg_v_negative_sampling(batch_size:int, n_neg_sample:int):
    """
    위에서 정의한 sample_table에서 (batch_size, n_neg_sample) shape만큼 랜덤 추출해 "네거티브 샘플 메트릭스"를 생성
    np.random.choice() 함수 활용 (위에서 정의한 sample_table을 함수의 argument로 사용)
    """
    neg_v = [np.random.choice(sample_table,  n_neg_sample) for _ in range(batch_size)]
    return neg_v

In [ ]:
get_neg_v_negative_sampling(4, 5)

[array([  30, 4782,   55, 2998, 2118]),
 array([ 292, 4503,  372, 3751, 1315]),
 array([ 257,  203,   66, 3731, 1704]),
 array([1635,  880, 3810,  908,  105])]

## Challenge

### 미니 튜토리얼
- 아래 튜토리얼을 따라하며 Skip-Gram 모델의 `forward` 및 `loss` 연산 방식을 이해하자
- Reference
    - [torch.nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)
    - [torch bmm](https://pytorch.org/docs/stable/generated/torch.bmm.html)
    - [Skip-Gram negative sampling loss function 설명 영문 블로그](https://aegis4048.github.io/optimize_computational_efficiency_of_skip-gram_with_negative_sampling#Derivation-of-Cost-Function-in-Negative-Sampling)
    - [Skip-Gram negative sampling loss function 설명 한글 블로그](https://reniew.github.io/22/)

In [ ]:
# hyper parameter example
emb_size = 30000 # vocab size
emb_dimension = 300 # word embedding 차원
n_neg_sample = 5
batch_size = 32

In [ ]:
# 1. Embedding Matrix와 Context Matrix를 생성
u_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)
v_embedding = nn.Embedding(emb_size, emb_dimension, sparse=True).to(device)

In [ ]:
# 2. wid(단어 인덱스)를 임의로 생성
pos_u = torch.randint(high = emb_size, size = (batch_size,))
pos_v = torch.randint(high = emb_size, size = (batch_size,))
neg_v = get_neg_v_negative_sampling(batch_size, n_neg_sample)
print(f"Target word idx : {pos_u} Pos context word idx : {pos_v} Neg context word idx : {neg_v}\n")

Target word idx : tensor([10412, 10342, 15354,  7414, 27369,  5960, 20544, 24658, 20770, 26139,
        21833, 12195, 22567,  2509, 24546,  5082, 13989, 19901, 21580, 17195,
        23444, 21426, 15964, 13286,  3684,  2218,  5672, 26348, 28218, 27557,
         5571, 21609]) Pos context word idx : tensor([16366, 22277,  5178, 12533,  2921, 13236, 23742, 18094,  1738, 13947,
        14945,  6577, 19916, 17750,   528, 23110,  2933, 21660, 27591, 20260,
         1512, 27951, 22080, 22420, 13676, 19662,  8944,  9258, 22481, 16151,
        11116,  9015]) Neg context word idx : [array([5187,  557, 1060,  190, 2697]), array([   5, 1298, 5160,  668, 1529]), array([  31,   97,   25, 1008, 1776]), array([4815, 5374,  775, 2981,  661]), array([2892,    3,   51, 2361, 3168]), array([1129, 1516, 4870,   38,  643]), array([5408, 5435, 1459,    9,  904]), array([5372,  891,  721, 2228, 5157]), array([ 298, 1180, 4261, 3451,  939]), array([2862, 4374,   54,  385, 2404]), array([2467, 1040,  764, 3346, 

In [ ]:
# 3. tensor로 변환
pos_u = Variable(torch.LongTensor(pos_u)).to(device)
pos_v = Variable(torch.LongTensor(pos_v)).to(device)
neg_v = Variable(torch.LongTensor(neg_v)).to(device)

In [ ]:
# 4. wid로 각각의 embedding matrix에서 word embedding 값을 가져오기
pos_u = u_embedding(pos_u)
pos_v = v_embedding(pos_v)
neg_v = v_embedding(neg_v)
print(f"shape of pos_u embedding : {pos_u.shape}\n shape of pos_v embedding : {pos_v.shape}\n shape of neg_v embedding : {neg_v.shape}")


shape of pos_u embedding : torch.Size([32, 300])
 shape of pos_v embedding : torch.Size([32, 300])
 shape of neg_v embedding : torch.Size([32, 5, 300])


In [ ]:
# 5. dot product 
pos_score = torch.mul(pos_u, pos_v) # 행렬 element-wise 곱
pos_score = torch.sum(pos_score, dim=1)
print(f"shape of pos logits : {pos_score.shape}\n")

neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)).squeeze()
print(f"shape of logits : {neg_score.shape}")

shape of pos logits : torch.Size([32])

shape of logits : torch.Size([32, 5])


In [ ]:
# 6. loss 구하기
pos_score = F.logsigmoid(pos_score)
neg_score = F.logsigmoid(-1*neg_score) # negative의 logit은 minimize 하기 위해 -1 곱함
print(f"pos logits : {pos_score.sum()}")
print(f"neg logits : {neg_score.sum()}")
loss = -1 * (torch.sum(pos_score) + torch.sum(neg_score))
print(f"Loss : {loss}")

pos logits : -231.58099365234375
neg logits : -1126.24169921875
Loss : 1357.82275390625


### Skip-gram 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `SkipGram` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()` 함수) 입력 매개변수
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `device` : 연산 장치 종류
    - 생성자에서 생성해야할 변수 
        - `vocab_size` : 사전내 단어 개수
        - `emb_dimension` : 엠베딩 크기
        - `u_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (target_word)
        - `v_embedding` : (vocab_size, emb_dimension) 엠베딩 메트릭스 (context_word)
    - 메소드
        - `init_embedding()` (제공됨)
            - 엠베딩 메트릭스 값을 초기화
        - `forward()`
            - 위 튜토리얼과 같이 dot product를 수행한 후 score를 생성
            - loss를 반환 (loss 설명 추가)
        - `save_emedding()` (제공됨)
            - `u_embedding`의 단어 엠베딩 값을 단어 별로 파일에 저장
    - 주의 사항     
        - `nn.Module`를 부모 클래스로 상속 받음 

In [ ]:
class SkipGram(nn.Module):
    def __init__(self, vocab_size:int, emb_dimension:int, device:str):
        super(SkipGram, self).__init__()
        self.vocab_size = vocab_size
        self.emb_dimension =  emb_dimension
        self.u_embedding =  nn.Embedding(self.vocab_size, self.emb_dimension, sparse=True).to(device)
        self.v_embedding = nn.Embedding(self.vocab_size, self.emb_dimension, sparse=True).to(device)
        self.init_embedding()
    
    
    def init_embedding(self):
        """
        u_embedding과 v_embedding 메트릭스 값을 초기화
        """
        initrange = 0.5 / self.emb_dimension
        self.u_embedding.weight.data.uniform_(-initrange, initrange)
        self.v_embedding.weight.data.uniform_(-0, 0)
    
    
    def forward(self, pos_u, pos_v, neg_v):
        """
        dot product를 수행한 후 score를 생성
        loss 반환
        """    
            
        # 각각의 embedding matrix에서 word embedding 값을 가져오기
        pos_u = self.u_embedding(pos_u),
        pos_v = self.v_embedding(pos_v),
        neg_v = self.v_embedding(neg_v)
        pos_u=pos_u[0]
        pos_v=pos_v[0]
        #neg_v=neg_v[0]

        # dot product 
        pos_score = torch.sum(torch.mul(pos_u, pos_v), dim=1)
        neg_score = torch.bmm(neg_v, pos_u.unsqueeze(dim=2)).squeeze()
        
        # loss 구하기
        loss = -1 * (torch.sum(F.logsigmoid(pos_score))+ torch.sum(F.logsigmoid(-1*neg_score)))

        return loss
    
    def save_embedding(self, id2word, file_name, use_cuda):
        """
        'file_name' 위치에 word와 word_embedding을 line-by로 저장
        파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
        """
        if use_cuda: # parameter를 gpu 메모리에서 cpu 메모리로 옮김
            embedding = self.u_embedding.weight.cpu().data.numpy()
        else:
            embedding = self.u_embedding.weight.data.numpy()

        with open(file_name, 'w') as writer:
            # 파일의 첫 줄은 '단어 개수' 그리고 '단어 embedding 사이즈' 값을 입력해야 함
            writer.write(f"{len(id2word)} {embedding.shape[-1]}\n")
            
            for wid, word in id2word.items():
                e = embedding[wid]
                e = " ".join([str(e_) for e_ in e])
                writer.write(f"{word} {e}\n")

## Advanced

### Skip-Gram 방식의  Word2Vec 클래스 구현
- Skip-Gram 방식으로 단어 embedding을 학습하는 `Word2Vec` 클래스를 구현하라.
- 클래스 정의
    - 생성자(`__init__()`) 입력 매개 변수
        - `input_file` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `device` : 연상 장치 종류
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `min_count` : 사전에 추가될 단어의 최소 등장 빈도
    - 생성자에서 생성해야 할 변수 
        - `docs` : 학습할 문서 리스트
        - `output_file_name` : 학습된 word embedding을 저장할 파일 위치
        - `word2count`, `word2id`, `id2word` : 위에서 구현한 `make_vocab()` 함수의 반환 값
        - `device` : 연산 장치 종류
        - `emb_size` : vocab의 (unique한) 단어 종류 
        - `emb_dimension` : word embedding 차원
        - `batch_size` : 학습 배치 사이즈
        - `window_size` : skip-gram 윈도우 사이즈 (context word 개수를 결정)
        - `n_neg_sample` : negative sample 개수
        - `iteration` : 학습 반복 횟수
        - `lr` : learning rate
        - `model` : `SkipGram` 클래스의 인스턴스
        - `optimizer` : `SGD` 클래스의 인스턴스
    - 메소드
        - `train()`
            - 입력 매개변수 
                - `train_dataloader`
            - Iteration 횟수만큼 input_file 학습 데이터를 학습한다. 매 epoch마다 for loop 돌면서 batch 단위 학습 데이터를 skip gram 모델에 학습함. 학습이 끝나면 word embedding을 output_file_name 파일에 저장.
- Reference
    - [Optimizer - SGD](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html)

In [ ]:
class Word2Vec:
    def __init__(self, 
                input_file: List[str],
                output_file_name: str,
                 device: str,
                 emb_dimension=300,
                 batch_size = 64,
                 window_size=5,
                 n_neg_sample = 5,
                 iteration=1,
                 lr = 0.02,
                 min_count=5):
        self.docs = input_file
        self.output_file_name = output_file_name
        self.word2count, self.word2id, self.id2word = make_vocab(self.docs, min_count )
        self.device = device
        self.emb_size = len(self.word2id)
        self.emb_dimension = emb_dimension
        self.batch_size = batch_size
        self.window_size = window_size
        self.n_neg_sample = n_neg_sample
        self.iteration = iteration
        self.lr = lr
        self.model = SkipGram(self.emb_size,self.emb_dimension,self.device)
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.lr) # torch.optim.SGD 클래스 사용

        # train() 함수에서 만든 임베딩 결과 파일들을 저장할 폴더 생성 (os.makedirs 사용)
        os.makedirs(self.output_file_name)
        
    
    def train(self, train_dataloader):
        
        # lr 값을 조절하는 스케줄러 인스턴스 변수를 생성
        self.scheduler = get_linear_schedule_with_warmup(
            optimizer = self.optimizer,
            num_warmup_steps=0,
            num_training_steps=len(train_dataloader)* self.iteration
        )
        
        for epoch in range(self.iteration):
            
            print(f"*****Epoch {epoch} Train Start*****")
            print(f"*****Epoch {epoch} Total Step {len(train_dataloader)}*****")
            total_loss, batch_loss, batch_step = 0,0,0

            for step, batch in enumerate(train_dataloader):
                batch_step+=1

                pos_u, pos_v = batch
                # negative data 생성
                neg_v = get_neg_v_negative_sampling(pos_u.shape[0], self.n_neg_sample)
                

                # 데이터를 tensor화 & device 설정
                pos_u = Variable(torch.LongTensor(pos_u)).to(device)
                pos_v = Variable(torch.LongTensor(pos_v)).to(device)
                neg_v = Variable(torch.LongTensor(neg_v)).to(device)

                # model의 gradient 초기화
                self.model.zero_grad()
                # optimizer의 gradient 초기화
                self.optimizer.zero_grad()

                # forward
                loss = self.model(pos_u,pos_v,neg_v)

                # loss 계산
                loss.backward()

                # optimizer 업데이트
                self.optimizer.step()
                # scheduler 업데이트
                self.scheduler.step()

                batch_loss += loss.item()
                total_loss += loss.item()
                
                if (step%500 == 0) and (step!=0):
                    print(f"Step: {step} Loss: {batch_loss/batch_step:.4f} lr: {self.optimizer.param_groups[0]['lr']:.4f}")
                    # 변수 초기화    
                    batch_loss, batch_step = 0,0
            
            print(f"Epoch {epoch} Total Mean Loss : {total_loss/(step+1):.4f}")
            print(f"*****Epoch {epoch} Train Finished*****\n")
            
            print(f"*****Epoch {epoch} Saving Embedding...*****")
            self.model.save_embedding(self.id2word, os.path.join(self.output_file_name, f'w2v_{epoch}.txt'), True if 'cuda' in self.device.type else False)
            print(f"*****Epoch {epoch} Embedding Saved at {os.path.join(self.output_file_name, f'w2v_{epoch}.txt')}*****\n")

In [ ]:
output_file = os.path.join(".", "word2vec_wiki")
# Word2Vec 클래스의 인스턴스 생성
w2v = Word2Vec(docs, output_file, device, n_neg_sample=10, iteration=3)

100%|██████████| 500/500 [01:18<00:00,  6.38it/s]


In [ ]:
# 학습 데이터 셋 및 데이터 로더 생성 (위에서 생성한 w2v의 attribute들을 argument에 적절히 넣기)
dataset = CustomDataset(docs, word2id, window_size=5)
train_dataloader = DataLoader(dataset, batch_size = 64, shuffle = True)
len(train_dataloader)

22171

In [ ]:
# 학습
w2v.train(train_dataloader)

*****Epoch 0 Train Start*****
*****Epoch 0 Total Step 22171*****
Step: 500 Loss: 487.6166 lr: 0.0198
Step: 1000 Loss: 448.6743 lr: 0.0197
Step: 1500 Loss: 373.6931 lr: 0.0195
Step: 2000 Loss: 323.1925 lr: 0.0194
Step: 2500 Loss: 289.8414 lr: 0.0192
Step: 3000 Loss: 268.3135 lr: 0.0191
Step: 3500 Loss: 251.8421 lr: 0.0189
Step: 4000 Loss: 240.4729 lr: 0.0188
Step: 4500 Loss: 231.7444 lr: 0.0186
Step: 5000 Loss: 225.7293 lr: 0.0185
Step: 5500 Loss: 221.2200 lr: 0.0183
Step: 6000 Loss: 218.1870 lr: 0.0182
Step: 6500 Loss: 215.2619 lr: 0.0180
Step: 7000 Loss: 212.6964 lr: 0.0179
Step: 7500 Loss: 210.7946 lr: 0.0177
Step: 8000 Loss: 208.9364 lr: 0.0176
Step: 8500 Loss: 207.7387 lr: 0.0174
Step: 9000 Loss: 206.4541 lr: 0.0173
Step: 9500 Loss: 205.7099 lr: 0.0171
Step: 10000 Loss: 204.8585 lr: 0.0170
Step: 10500 Loss: 204.0793 lr: 0.0168
Step: 11000 Loss: 203.4739 lr: 0.0167
Step: 11500 Loss: 202.5938 lr: 0.0165
Step: 12000 Loss: 202.1092 lr: 0.0164
Step: 12500 Loss: 201.7606 lr: 0.0162
Step:

### 유사한 단어 확인
- 사전에 존재하는 단어들과 유사한 단어를 검색해보자. Gensim 패키지는 유사 단어 외에도 단어간의 유사도를 계산하는 여러 함수를 제공한다. 실험을 통해 word2vec의 한계점을 발견했다면 아래에 markdown으로 작성해보자. 
- [Gensim 패키지 document](https://radimrehurek.com/gensim/models/keyedvectors.html)

In [ ]:
import gensim

In [ ]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/drive/MyDrive/MoneyManager/report/tokenized/word2vec_wiki/w2v_0.txt', binary=False)

In [ ]:
word_vectors.most_similar(positive=['공주'])

[('약칭', 0.9975637197494507),
 ('메인', 0.9972103834152222),
 ('임시', 0.9971497654914856),
 ('학력', 0.9969079494476318),
 ('절망선생', 0.9966758489608765),
 ('본국', 0.9966574907302856),
 ('학과', 0.9965877532958984),
 ('학부', 0.9965819716453552),
 ('대검찰청', 0.9965457320213318),
 ('탈해', 0.9965407848358154)]

word2vec의 한계점은?

- 단어의 형태학적 특성을 반영하지 못한다는 점
 : 학생, 학생들 같이 의미가 유사하지만 개별적으로 Embedding하기 때문에 Vector가 유사하게 구성되지 않습니다.
- 희소한 단어를 Embedding 하기 어렵다는 점
: 분포가설 기반으로 학습하기에 출현횟수가 많은 단어에 대해서는 잘 임베딩하나 적은 단어의 경우 제대로 임베딩 되지가 않습니다.
- OOV 를 처리할 수 없다는 한계점이 있다
:  단어단위로 어휘집(Vocabulary)를 구성하기 때문에, 어휘집에 없는 새로운 단어가 등장하면 데이터 전체를 다시 학습시켜야 합니다.